In [2]:
import findspark
findspark.init()

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [4]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("ReadFromFile") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory", "4g") \
.getOrCreate()

In [5]:
schema = StructType([
StructField("sirano", IntegerType(), True),
StructField("isim", StringType(), True),
StructField("yas", IntegerType(), True),
StructField("meslek", StringType(), True),
StructField("sehir", StringType(), True),
StructField("aylik_gelir", DoubleType(), True)
])

In [6]:
# D:\\spark-streaming-test dizininden simple_data.csv dosyasını monitör edeceğiz
df = spark.readStream \
.format("csv") \
.option("header",True) \
.option("sep",",") \
.schema(schema) \
.load("D:\\spark-streaming-test")

In [10]:
meslekOrtGelir = df.groupBy("meslek") \
.agg(f.avg("aylik_gelir").alias("ort_gelir")) \
.sort(f.desc("ort_gelir"))

In [11]:
query = meslekOrtGelir.writeStream \
      .outputMode("complete") \
      .format("console") \
      .start()

In [ ]:
query.awaitTermination()

### Monitör edilen dizine text dosyasını kopyala ve konsoldan sonçları gözlemle.
    +-----------+------------------+
    |     meslek|         ort_gelir|
    +-----------+------------------+
    |     Doktor|           16125.0|
    |     Berber|           12000.0|
    |   Müzisyen|            9900.0|
    |Pazarlamaci| 5433.333333333333|
    | Tuhafiyeci|            4800.0|
    |    Tornacı|            4200.0|
    |      Memur|4066.6666666666665|
    |       Isci|            3500.0|
    +-----------+------------------+

In [ ]:
query.stop()